<a href="https://colab.research.google.com/github/Tanishqchahal/NYC_Airbnb_Market/blob/main/NYC_Airbnb_Market.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import datetime as dt

In [ ]:
prices = pd.read_csv('airbnb_price.csv')
room_typ = pd.read_excel('airbnb_room_type.xlsx')
reviews = pd.read_csv('airbnb_last_review.tsv', sep='\t')

In [ ]:
room_typ.drop(columns=['Unnamed: 0'],inplace=True)

In [ ]:
prices.head()

,listing_id,price,nbhood_full
0,2595,225 dollars,"Manhattan, Midtown"
1,3831,89 dollars,"Brooklyn, Clinton Hill"
2,5099,200 dollars,"Manhattan, Murray Hill"
3,5178,79 dollars,"Manhattan, Hell's Kitchen"
4,5238,150 dollars,"Manhattan, Chinatown"


In [ ]:
prices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25209 entries, 0 to 25208
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   listing_id   25209 non-null  int64 
 1   price        25209 non-null  object
 2   nbhood_full  25209 non-null  object
dtypes: int64(1), object(2)
memory usage: 591.0+ KB


In [ ]:
room_typ.head()

,listing_id,room_type,number_of_reviews
0,2595,Entire home/apt,48
1,3831,Entire home/apt,295
2,5099,Entire home/apt,78
3,5121,Private room,49
4,5178,Private room,454


In [ ]:
reviews.head()

,listing_id,host_name,last_review
0,2595,Jennifer,May 21 2019
1,3831,LisaRoxanne,July 05 2019
2,5099,Chris,June 22 2019
3,5178,Shunichi,June 24 2019
4,5238,Ben,June 09 2019


In [ ]:
room_typ.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17614 entries, 0 to 17613
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   listing_id         17614 non-null  int64 
 1   room_type          17614 non-null  object
 2   number_of_reviews  17614 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 413.0+ KB


In [ ]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25209 entries, 0 to 25208
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   listing_id   25209 non-null  int64 
 1   host_name    25201 non-null  object
 2   last_review  25209 non-null  object
dtypes: int64(1), object(2)
memory usage: 591.0+ KB


In [ ]:
prices['price'] = prices['price'].str.strip('dollars').astype(int)

In [ ]:
prices = prices[prices['price']!=0]

In [ ]:
avg_price = prices['price'].mean()
print(f'Average price of Airbnb rentals is {avg_price:0.2f}')

Average price of Airbnb rentals is 141.82


##Comparing Costs to Private market

According to Zumper, a 1 bedroom apartment in New York City costs, on average, $3,100 per month. Let's convert the per night prices of our listings into monthly costs, so we can compare to the private market.

In [ ]:
prices['price_per_month'] = prices['price']* 365/12

<ipython-input-70-447aad084ebb>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prices['price_per_month'] = prices['price']* 365/12


In [ ]:
avg_price_per_month = prices['price_per_month'].mean()
print(f'Airbnb monthly costs are {avg_price_per_month:.2f}, while in private markets you would pay around $3100.')

Airbnb monthly costs are 4313.61, while in private markets you would pay around $3100.


In [ ]:
room_typ['room_type'].value_counts()

room_type
Entire home/apt    9405
Private room       7752
Shared room         357
Hotel room          100
Name: count, dtype: int64

In [ ]:
reviews['last_review'] = pd.to_datetime(reviews['last_review'])

In [ ]:
earliest_review = reviews['last_review'].dt.date.min()
last_review = reviews['last_review'].dt.date.max()
print(f'The latest Airbnb review is {last_review}, the earliest review is {earliest_review}')

The latest Airbnb review is 2019-07-09, the earliest review is 2019-01-01


##Merge Dataframes

In [ ]:
rooms_and_prices = pd.merge(room_typ,prices,how='outer',on='listing_id')

In [ ]:
airbnb_data = pd.merge(rooms_and_prices,reviews,how='outer',on='listing_id')

In [ ]:
airbnb_data.head()

,listing_id,room_type,number_of_reviews,price,nbhood_full,price_per_month,host_name,last_review
0,2595,Entire home/apt,48.0,225.0,"Manhattan, Midtown",6843.750000,Jennifer,2019-05-21
1,3831,Entire home/apt,295.0,89.0,"Brooklyn, Clinton Hill",2707.083333,LisaRoxanne,2019-07-05
2,5099,Entire home/apt,78.0,200.0,"Manhattan, Murray Hill",6083.333333,Chris,2019-06-22
3,5121,Private room,49.0,NaN,NaN,NaN,NaN,NaT
4,5178,Private room,454.0,79.0,"Manhattan, Hell's Kitchen",2402.916667,Shunichi,2019-06-24


In [ ]:
airbnb_data.dropna(inplace=True)

In [82]:
airbnb_data['borough'] = airbnb_data['nbhood_full'].str.partition(",", expand=True)[0]

In [84]:
boroughs = airbnb_data.groupby('borough')['price'].agg(['sum','mean','median','count'])
boroughs = boroughs.round(2).sort_values('mean',ascending=False)
boroughs

,sum,mean,median,count
borough,,,,
Manhattan,893869.0,171.70,139.0,5206
Brooklyn,742816.0,123.02,100.0,6038
Queens,174429.0,92.73,73.0,1881
Staten Island,13439.0,86.15,70.5,156
Bronx,30954.0,81.03,63.5,382
